Question: Which words / phrases appear more in toxic content?

In [30]:
import pandas as pd
import time

df = pd.read_csv('Sample_labaled_data.csv')

In [11]:
print(df)

       Unnamed: 0                id  \
0               5  0001ea8717f6de06   
1               7  000247e83dcc1211   
2              11  0002f87b16116a7f   
3              13  0003e1cccfd5a40a   
4              14  00059ace3e3e9a53   
...           ...               ...   
55247      153147  fff83b80284d8440   
55248      153149  fff8f521a7dbcd47   
55249      153150  fff8f64043129fa2   
55250      153151  fff9d70fe0722906   
55251      153154  fffa8a11c4378854   

                                            comment_text toxic  
0      Thank you for understanding I think very highl...    no  
1                         Dear god this site is horrible    no  
2       Somebody will invariably try to add Religion ...    no  
3        It says it right there that it IS a type The...    no  
4         Before adding a new product to the list mak...    no  
...                                                  ...   ...  
55247  Consensus for ruining Wikipedia I think that c...    no  
55248  shut

This is the function that I will utilize to request the Perspective API for my personal evaluation. I created my API key and incorporated it into API_KEY.

In [33]:
from googleapiclient.discovery import build
import json

def get_toxicity_score(comment):
    
  API_KEY = 'my-api-key' # Put your API key here
    
  client = build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  )

  analyze_request = {
  'comment': { 'text': comment },
  'requestedAttributes': {'TOXICITY': {}}
  }
    
  response = client.comments().analyze(body=analyze_request).execute()
  toxicity_score = response["attributeScores"]["TOXICITY"]["summaryScore"]["value"]
    
  return toxicity_score

Upon analyzing the data, I hypothesized that a comment was considered toxic if it used female family gendered pronouns such as mom, mother, etc. in comparison to male family gendered pronouns such as dad, father, etc. 

In [37]:
comment_list = ['ur mom', 'ur dad', 'ur moms ugly', 'ur dads ugly', 'ur mom is rachet', 'ur dad is rachet', 'ur moms a hoe', 'ur dads a hoe', 'i love hot moms', 'i love hot dads', 'thats why ur mom left you', 'thats why ur dad left u', 'mom', 'dad', 'mommy', 'daddy', 'mother','father','ur mother', 'ur father']
score_list = []

for comment in comment_list:
    score = get_toxicity_score(comment)
    score_list.append(score)
    print(comment, score)
    time.sleep(1)

ur mom 0.28425202
ur dad 0.15435068
ur moms ugly 0.8364697
ur dads ugly 0.78207105
ur mom is rachet 0.5024724
ur dad is rachet 0.43720075
ur moms a hoe 0.8460273
ur dads a hoe 0.82048255
i love hot moms 0.4269174
i love hot dads 0.4402136
thats why ur mom left you 0.36867723
thats why ur dad left u 0.30136615
mom 0.039935954
dad 0.02826718
mommy 0.044388983
daddy 0.04711028
mother 0.083999306
father 0.02638162
ur mother 0.3250515
ur father 0.14429577


Based on my tests, I have found that my hypothesis was accurate. The Perspective API evaluated identical comments differently when the terms 'mom' or 'mommy' were replaced with 'dad' or 'daddy'. In every case, comments that referred to a female parent received a higher toxicity score compared to those referring to a male parent.